<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/3_Limpieza_datos_MetaCortex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Limpieza de datos y tratamiento de los valores nulos

El objetivo es limpiar y estandarizar el dataset de empleados de MetaCortex (tratamiento de nulos y duplicados en `nif`, inferencia/completado de `genero` y `sexo_biologico`, normalización de `activo` y `horas_semanales`, imputación de `pago` y `bonus`, selección/renombrado de columnas clave) y exportar el dataset final.

Cargamos el dataset previamente tratado:

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
import pandas as pd
df = pd.read_feather('cdiae/data/02_intermediate/empleados_metacortex.ftr')
df

In [ ]:
df.info()

## Tratamiento de valores nulos
Utilizaremos las siguientes funciones para operar con los valores nulos:
- isnull()
- dropna()
- duplicated()
- drop_duplicates()

In [ ]:
df.isnull().sum()       # Conteo de valores nulos por columna

### Tratamiento de valores nulos: Borrado
Por ejemplo, el valor nif es un dato que no puede inferirse, y a efectos de reporte es imprescindible al ser la clave única. En este caso, la opción va a ser eliminar el registro completo

In [ ]:
df.dropna(subset=['nif'], inplace=True) # Elimina filas con nif nulo (clave única)
df.isnull().sum()                       # Recuento de nulos tras eliminar nif nulos

Si el valor es único (como nif) busca duplicados:

In [ ]:
df_test = df[df['nif'].duplicated(keep=False)]
df_test

Valores posibles del parámetro **keep**:
- `keep='first'` (por defecto)
- `keep='last'`
- `keep=False`

La función `drop_duplicates()` funciona en el mismo sentido eliminandolos en vez de seleccionandolos.

En nuestros datos, vamos a dejar la última ocurrencia por nif al entender que es la más reciente:

In [ ]:
df.drop_duplicates(subset=['nif'], keep='last', inplace=True)
df

## Rellenando los valores nulos o vacíos

In [ ]:
df['genero'] = df['genero'].fillna(df.sexo_biologico)
df

Nota: Existen dos librerías muy utilizadas para inferir el genero partiendo del nombre de pila: **gender_guesser** y **names_dataset**.

La función **fillna** dispone de los siguientes métodos para rellenar:

- `df.fillna(method='ffill')`      # Forward fill (propaga hacia adelante)
- `df.fillna(method='bfill')`      # Backward fill (propaga hacia atrás)
- `df.fillna(method='pad')`        # Igual que ffill
- `df.fillna(method='backfill')`

Tambien podemos definir límites:
- `df.fillna(method='ffill', limit=2)`


In [ ]:
df['pago'] = df['pago'].fillna(0)       # 0 si está vacio
df

Creamos el campo 'bonus1' con la media global de 'bonus'

In [ ]:
df['bonus1'] = df['bonus'].fillna(df['bonus'].mean())
df

Creamos el campo 'bonus2' con la media por departamento

In [ ]:
df['bonus2'] = df.groupby('departamento')['bonus'].transform(
    lambda x: x.fillna(x.mean())
)
df

In [ ]:
df.isnull().sum()               # Verifica nulos pendientes

In [ ]:
df

Reseteo el index eliminado el index anterior

In [ ]:
df.reset_index(drop=True)

Convierte 'horas_semanales' de texto (e.g. '40h') a entero

In [ ]:
df['horas_semanales'] = df['horas_semanales'].str.rstrip('h').astype(int)
df

In [ ]:
# Verifica nulos pendientes
df.isnull().sum()

Seleccionamos y renombramos las columnas para el dataset final

In [ ]:
def selector_columnas(df: pd.DataFrame) -> pd.DataFrame:
    # Columnas a conservar
    cols_keep = [
        'nif',
        'fecha_nacimiento',
        'fecha_alta',
        'sexo_biologico',
        'activo',
        'codigo_postal',
        'bonus2',
    ]
    df = df[cols_keep]
    # Renombra a nombres de salida
    df = df.rename(columns={
        'nif':'IdAsegurado',
        'fecha_nacimiento':'FNacim',
        'fecha_alta':'FAlta',
        'sexo_biologico':'Sexo',
        'activo':'Activo',
        'codigo_postal':'CP',
        'bonus2':'Capital',
    })
    return df

In [ ]:
df = selector_columnas(df)
df

Exportamos el dataset limpio en formato Feather a la carpeta "03_model_input"

In [ ]:
df.to_feather('cdiae/data/03_model_input/empleados_metacortex_clean.ftr')